# <font color=black> Analysis - FC-based parcellation using mutual information </font>
<hr style="border:1px solid black">

In [1]:
import sys,json
import glob, os
import numpy as np
import nibabel as nib
import time

sys.path.append('/media/miplab-nas2/Data3/BMPD/hc_project/analysis/code/')

from fc_based_parcellation import FC_Parcellation

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Initialization `MOTOR`
<hr style="border:0.5px solid black">

### Load configuration file 

In [6]:
# Load the config file with all information
with open('../../config/config_fc_based_parcellation_NK.json') as config_file:
    config = json.load(config_file)

### Set relevant parameters

In [10]:
# Define tag of this analysis (used for saving)
config['output_tag'] = "frost_C3_gm_to_SMC"
# Define source, target, and the corresponding mask
source = 'spinalcord'
target = 'brain'

config['file_tag']['brain'] = "_task-rest_bold_stc_brain_moco_brsc_BP_reg_SPM_inTemplate_s.nii.gz"
config['file_tag']['spinalcord'] = "_task-rest_bold_stc_sc_moco_brsc_BP_inTemplate.nii"

# Note: paths should be with respect to main directory
config['masks']['brain'] = "/analysis/mask/brain/MNI_GM_3mm_SMC.nii.gz"
config['masks']['spinalcord'] = "/analysis/mask/spinalcord/levels/frostell/gm/frost_C3_gm.nii.gz"
config['list_subjects'] = ["P030", "P099","A006","A007","A008","A009","A011","A013","A023","A024","A025","A026","A027","A028","A029","A030","A032","A033","A034","A036","A037","A038","A039","A043","A044","A045","A046","A048","A050","A051","A052"]

### Instantiate class
_Note: this will save a config file with all relevant information and prepare folder structure_

In [11]:
fc_parcellation = FC_Parcellation(config,source,target,'mi')

# Analyses
<hr style="border:0.5px solid black">

### Compute the voxelwise correlation & run clustering for each subject independently

In [12]:
start = time.time()
# Loop through the subjects to compute fc and clustering 
for sub_id,sub in enumerate(fc_parcellation.config['list_subjects']): 
    print(f'\033[1mSUBJECT {sub}\033[0m')
    # Compute correlation without saving files
    fc_parcellation.compute_voxelwise_fc(sub=sub, 
                                        overwrite=True)
    # Run clustering and save labels
    fc_parcellation.run_clustering(sub=sub,
                                    k_range=4,algorithm='kmeans',
                                    overwrite=False)
print("\033[1m... Operation performed in %.2f s!\033[0m" % (time.time() - start))

SUBJECT P030
COMPUTE VOXELWISE FC
Standardize = True
Overwrite results = True
... Loading data
... Computing FC for all possibilities
... Metric: mutual information


  0%|                                                                                                                            | 0/1880 [00:00<?, ?it/s]


PicklingError: Can't pickle <class 'fc_based_parcellation.FC_Parcellation'>: it's not the same object as fc_based_parcellation.FC_Parcellation

### Compute the group-level clustering and generate corresponding maps

In [ ]:
fc_parcellation.group_clustering(k_range=4,indiv_algorithm='kmeans',overwrite=False)

# Initialization `SENSORY`
<hr style="border:0.5px solid black">

### Load configuration file 

In [ ]:
# Load the config file with all information
with open('../../config/config_fc_based_parcellation_NK_BRtoSC.json') as config_file:
    config = json.load(config_file)

### Set relevant parameters

In [ ]:
# Define tag of this analysis (used for saving)
config['output_tag'] = "postcentral_sensory_to_dorsal"
# Define source, target, and the corresponding mask
source = 'brain'
target = 'spinalcord'
# Note: paths should be with respect to main directory
config['masks'][source] = "/templates/MNI_hc_project/julich_postcentral_sensory.nii.gz"
config['masks'][target] = "/templates/PAM50_hc_project/template/PAM50_cord_C1toC8_D.nii.gz"
config['list_subjects'] = ["P099","A023","A024","A025","A026","A027","A028","A029","A030","A033","A034","A036","A037","A038","A039","A043"]

### Instantiate class
_Note: this will save a config file with all relevant information and prepare folder structure_

In [ ]:
fc_parcellation = FC_Parcellation(config,source,target,'corr')

# Analyses
<hr style="border:0.5px solid black">

### Compute the voxelwise correlation & run clustering for each subject independently

In [ ]:
start = time.time()
# Initialize empty array for individual labels
mask_source_path = config['main_dir']+config['masks'][source]
nvox_source = np.count_nonzero(nib.load(mask_source_path).get_fdata().astype(bool))
subject_labels = np.zeros((len(fc_parcellation.config['list_subjects']),nvox_source))
for sub_id,sub in enumerate(fc_parcellation.config['list_subjects']): 
    print(f'SUBJECT {sub}')
    # Compute correlation without saving files
    dict_corr = fc_parcellation.compute_voxelwise_fc(sub=sub, 
                                                  load_from_file=True, 
                                                  save_results=True, 
                                                  mask_source_path=mask_source_path, 
                                                  mask_target_path=config['main_dir']+config['masks'][target])
    # Run clustering and save labels
    subject_labels[sub_id,:] = fc_parcellation.run_clustering(dict_corr,
                                                              k=8,algorithm='kmeans',
                                                              load_from_file=True,
                                                              save_results=True)
print("... Operation performed in %.2f s!" % (time.time() - start))

### Compute the group-level clustering and generate corresponding maps

In [ ]:
group_labels, indiv_labels_relabeled = fc_parcellation.group_clustering(subject_labels)
fc_parcellation.prepare_seed_map(group_labels)
fc_parcellation.prepare_target_maps(indiv_labels_relabeled, load_from_file=True, save_results=True)